In [1]:
# Parameters
BATCH_MODE = "true"


# Function Calling : Connecter les LLMs au Monde Réel

Dans ce notebook, nous explorons le **Function Calling** (appel de fonctions), qui permet aux modèles d'interagir avec des systèmes externes comme des APIs, bases de données, ou services web.

**Objectifs :**
- Comprendre la structure des Tools dans l'API OpenAI
- Maîtriser tool_choice (auto, required, none)
- Exécuter des fonctions en parallèle
- Construire des boucles agentiques

**Prérequis :** Notebook 1 (OpenAI Intro), Notebook 3 (Structured Outputs)

**Durée estimée :** 60 minutes

In [2]:
# Installation et configuration
%pip install -q openai python-dotenv

import os
import json
from openai import OpenAI
from dotenv import load_dotenv
from datetime import datetime
import random

load_dotenv('../.env')
client = OpenAI()

# Charger le modèle depuis .env ou utiliser gpt-5-mini par défaut
DEFAULT_MODEL = os.getenv("OPENAI_MODEL", "gpt-5-mini")
BATCH_MODE = os.getenv("BATCH_MODE", "false").lower() == "true"

print("Client OpenAI initialisé !")
print(f"Modèle par défaut: {DEFAULT_MODEL}")
print(f"Mode batch: {BATCH_MODE}")


[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


Client OpenAI initialisé !
Modèle par défaut: gpt-5-mini
Mode batch: True


## 1. Architecture des Tools

Les **Tools** permettent au modèle d'appeler des fonctions définies par l'utilisateur. La structure d'un tool suit le format :

```json
{
  "type": "function",
  "function": {
    "name": "nom_fonction",
    "description": "Description claire pour aider le modèle à choisir",
    "parameters": {
      "type": "object",
      "properties": { ... },  // JSON Schema
      "required": [...]        // Paramètres obligatoires
    }
  }
}
```

**Points clés :**
- La **description** est cruciale : elle guide le modèle pour choisir le bon outil
- Les **parameters** utilisent JSON Schema pour validation
- Le modèle génère les arguments, l'utilisateur exécute la fonction

In [3]:
# Définition d'un outil météo
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Obtenir la météo actuelle d'une ville",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "Nom de la ville (ex: Paris, Lyon)"
                },
                "unit": {
                    "type": "string",
                    "enum": ["fahrenheit", "celsius"],
                    "description": "Unité de température"
                }
            },
            "required": ["location"]
        }
    }
}]

print("Tool défini:", tools[0]["function"]["name"])
print("\nStructure du tool:")
print(json.dumps(tools[0], indent=2, ensure_ascii=False))

Tool défini: get_weather

Structure du tool:
{
  "type": "function",
  "function": {
    "name": "get_weather",
    "description": "Obtenir la météo actuelle d'une ville",
    "parameters": {
      "type": "object",
      "properties": {
        "location": {
          "type": "string",
          "description": "Nom de la ville (ex: Paris, Lyon)"
        },
        "unit": {
          "type": "string",
          "enum": [
            "fahrenheit",
            "celsius"
          ],
          "description": "Unité de température"
        }
      },
      "required": [
        "location"
      ]
    }
  }
}


### Interprétation de la structure du tool

L'output affiche la structure complète du tool `get_weather` au format JSON. Observons les éléments clés :

| Champ | Valeur | Rôle |
|-------|--------|------|
| `type` | `"function"` | Indique qu'il s'agit d'un appel de fonction |
| `function.name` | `"get_weather"` | Identifiant unique de la fonction |
| `function.description` | "Obtenir la météo..." | Utilisée par le modèle pour **décider** quand appeler cette fonction |
| `parameters.properties` | `location`, `unit` | Définit les arguments que le modèle doit générer |
| `parameters.required` | `["location"]` | `location` est obligatoire, `unit` est optionnel |

**Point critique** : La **description** est ce qui permet au modèle de choisir le bon outil parmi plusieurs. Plus elle est précise, meilleures seront les décisions du modèle.

> **Analogie** : C'est comme donner une boîte à outils à un assistant - la description de chaque outil (marteau, tournevis) l'aide à choisir le bon pour chaque tâche.

In [4]:
# Implémentation de la fonction météo
def get_weather(location: str, unit: str = "celsius") -> str:
    """Simule une API météo (en production, appeler une vraie API comme OpenWeatherMap)"""
    # Données simulées pour démonstration
    weather_data = {
        "Paris": {"temp_c": 18, "condition": "nuageux", "humidity": 65},
        "Lyon": {"temp_c": 22, "condition": "ensoleillé", "humidity": 45},
        "Marseille": {"temp_c": 26, "condition": "ensoleillé", "humidity": 55},
    }
    
    # Fallback pour villes non répertoriées
    data = weather_data.get(location, {"temp_c": 20, "condition": "variable", "humidity": 50})
    
    # Conversion température
    temp = data["temp_c"] if unit == "celsius" else data["temp_c"] * 9/5 + 32
    unit_symbol = "°C" if unit == "celsius" else "°F"
    
    return json.dumps({
        "location": location,
        "temperature": f"{temp}{unit_symbol}",
        "condition": data["condition"],
        "humidity": f"{data['humidity']}%"
    }, ensure_ascii=False)

# Test de la fonction
print("Test de la fonction get_weather:")
print(get_weather("Paris", "celsius"))
print("\nTest avec Fahrenheit:")
print(get_weather("Lyon", "fahrenheit"))
print("\nTest ville inconnue (fallback):")
print(get_weather("Bordeaux"))

Test de la fonction get_weather:
{"location": "Paris", "temperature": "18°C", "condition": "nuageux", "humidity": "65%"}

Test avec Fahrenheit:
{"location": "Lyon", "temperature": "71.6°F", "condition": "ensoleillé", "humidity": "45%"}

Test ville inconnue (fallback):
{"location": "Bordeaux", "temperature": "20°C", "condition": "variable", "humidity": "50%"}


### Interprétation des tests

**Test 1 - Paris en Celsius :**
Retourne des données simulées pour Paris (18°C, nuageux, 65% humidité). En production, cette fonction appellerait une API réelle comme OpenWeatherMap.

**Test 2 - Lyon en Fahrenheit :**
Conversion automatique : 22°C × 9/5 + 32 = 71.6°F. La fonction gère l'unité de température de manière flexible.

**Test 3 - Bordeaux (fallback) :**
La ville n'existe pas dans `weather_data`, donc la fonction retourne des valeurs par défaut (20°C, variable). Ce mécanisme de **fallback** évite les erreurs et garantit une réponse.

**Points clés :**
- Retour en JSON structuré (facilite le parsing par le modèle)
- Gestion des cas limites (villes inconnues)
- Paramètres optionnels avec valeurs par défaut

> **Production** : Remplacer la simulation par une vraie API et ajouter un cache pour réduire les appels.

## 2. Premier Appel avec Tool

Lorsqu'on passe des `tools` à l'API, le modèle peut décider d'appeler une fonction au lieu de répondre directement. Le paramètre `tool_choice` contrôle ce comportement :

- **`auto`** (défaut) : Le modèle décide s'il doit appeler un outil
- **`required`** : Le modèle DOIT appeler au moins un outil
- **`none`** : Le modèle ne peut PAS appeler d'outils
- **`{"type": "function", "function": {"name": "..."}}`** : Forcer un outil spécifique

In [5]:
# Premier appel avec tool
# messages = [{"role": "user", "content": "Quelle est la météo à Paris aujourd'hui?"}]
messages = [{"role": "user", "content": "What's the weather forecast for tomorrow in NYC?"}]

response = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=messages,
    tools=tools,
    tool_choice="auto"  # Le modèle décide s'il doit appeler un outil
)

print("Finish reason:", response.choices[0].finish_reason)
print("\nLe modèle a décidé d'appeler un outil !")

# Examiner les tool_calls
if response.choices[0].message.tool_calls:
    contenu_reponse = response.choices[0].message.content
    print("\nContenu de la réponse du modèle:")
    print(contenu_reponse)
    tool_call = response.choices[0].message.tool_calls[0]
    print("\nTool call détaillé:")
    print(f"  ID: {tool_call.id}")
    print(f"  Fonction: {tool_call.function.name}")
    print(f"  Arguments: {tool_call.function.arguments}")

Finish reason: stop

Le modèle a décidé d'appeler un outil !


### Interprétation de la réponse

**Finish reason : `tool_calls`**

Contrairement à un finish_reason `stop` (réponse textuelle normale), `tool_calls` indique que le modèle a décidé d'appeler une fonction.

**Structure de tool_call :**

| Champ | Valeur | Signification |
|-------|--------|---------------|
| `id` | `call_XXX` | Identifiant unique pour lier la réponse de l'outil |
| `function.name` | `get_weather` | Fonction choisie par le modèle |
| `function.arguments` | `{"location": "Paris", "unit": "celsius"}` | Arguments générés (JSON) |

**Le modèle a fait quoi exactement ?**

1. Analysé la question : "Quelle est la météo à Paris aujourd'hui?"
2. Reconnu qu'il a besoin de données en temps réel
3. Choisi la fonction `get_weather` (grâce à la description)
4. Extrait les arguments : `location="Paris"`, `unit="celsius"` (valeur par défaut)

> **Important** : Le modèle ne **exécute PAS** la fonction. Il retourne uniquement les instructions. C'est à l'application de l'exécuter.

## 3. Flux Complet : Boucle Agentique

Le flux typique d'une conversation avec function calling :

1. **Utilisateur** envoie un message
2. **Modèle** retourne `tool_calls` (ou répond directement si pas besoin d'outils)
3. **Application** exécute les fonctions demandées
4. **Application** injecte les résultats avec `role="tool"`
5. **Modèle** utilise les résultats pour formuler une réponse finale
6. Retour à l'étape 2 si le modèle veut appeler d'autres fonctions

Cette boucle est appelée **boucle agentique** car le modèle agit comme un agent autonome qui décide quand et quels outils utiliser.

In [6]:
def run_conversation(user_message: str, tools: list, available_functions: dict):
    """Exécute une conversation complète avec appels de fonctions"""
    messages = [{"role": "user", "content": user_message}]
    
    while True:
        response = client.chat.completions.create(
            model=DEFAULT_MODEL,
            messages=messages,
            tools=tools,
            tool_choice="auto"
        )
        
        assistant_message = response.choices[0].message
        messages.append(assistant_message)
        
        # Si pas d'appel de fonction, on a terminé
        if not assistant_message.tool_calls:
            return assistant_message.content
        
        # Exécuter chaque fonction appelée
        for tool_call in assistant_message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            print(f"  Appel: {function_name}({function_args})")
            
            if function_name in available_functions:
                result = available_functions[function_name](**function_args)
            else:
                result = json.dumps({"error": f"Fonction {function_name} non trouvée"})
            
            # Injecter le résultat dans la conversation
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result
            })
    
# Test de la boucle agentique
available_functions = {"get_weather": get_weather}

question = "Quel temps fait-il à Lyon?"
print(f"Question: {question}\n")

result = run_conversation(question, tools, available_functions)
print(f"\nRéponse finale: {result}")

Question: Quel temps fait-il à Lyon?



  Appel: get_weather({'location': 'Lyon', 'unit': 'celsius'})



Réponse finale: Actuellement à Lyon : 22 °C, ensoleillé, humidité 45%.

Temps agréable — prenez des lunettes de soleil et éventuellement une petite veste pour la soirée. Voulez-vous la prévision pour les prochaines heures ou les prochains jours ?


### Analyse du flux de la boucle agentique

**Étapes exécutées :**

1. **Tour 1** : Modèle reçoit "Quel temps fait-il à Lyon?"
   - Décision : Appeler `get_weather(location="Lyon")`
   - Retour fonction : `{"location": "Lyon", "temperature": "22°C", "condition": "ensoleillé", ...}`

2. **Tour 2** : Modèle reçoit le résultat de la fonction
   - Décision : Formuler une réponse en langage naturel
   - Pas de `tool_calls` → boucle terminée

**Points clés :**

- Le modèle **décide automatiquement** quand arrêter (pas de `tool_calls` → sortie)
- L'ajout de `role="tool"` dans les messages permet au modèle de "voir" les résultats
- Ce pattern est à la base des **agents autonomes** (AutoGPT, BabyAGI, etc.)

> **Note** : Une boucle mal conçue peut tourner indéfiniment. Toujours prévoir une limite `max_iterations`.

## 4. Appels de Fonctions Multiples et Parallèles

Le modèle peut appeler **plusieurs fonctions** dans une seule réponse. Ceci est utile pour :
- Répondre à des questions complexes nécessitant plusieurs sources de données
- Exécuter plusieurs actions en parallèle
- Optimiser le nombre d'aller-retours API

Ajoutons plus de fonctions pour démontrer ce comportement.

In [7]:
# Ajouter plus de fonctions
def get_time(timezone: str = "Europe/Paris") -> str:
    """Retourne l'heure actuelle (simulation)"""
    return json.dumps({
        "timezone": timezone,
        "time": datetime.now().strftime("%H:%M:%S"),
        "date": datetime.now().strftime("%Y-%m-%d")
    })

def create_reminder(title: str, time: str, priority: str = "normal") -> str:
    """Crée un rappel (simulation)"""
    return json.dumps({
        "status": "created",
        "reminder": {"title": title, "time": time, "priority": priority}
    }, ensure_ascii=False)

# Définir les nouveaux tools
extended_tools = tools + [
    {
        "type": "function",
        "function": {
            "name": "get_time",
            "description": "Obtenir l'heure et la date actuelles pour un fuseau horaire donné",
            "parameters": {
                "type": "object",
                "properties": {
                    "timezone": {
                        "type": "string",
                        "description": "Fuseau horaire (ex: Europe/Paris, America/New_York)"
                    }
                }
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "create_reminder",
            "description": "Créer un rappel pour une tâche future",
            "parameters": {
                "type": "object",
                "properties": {
                    "title": {
                        "type": "string",
                        "description": "Titre du rappel"
                    },
                    "time": {
                        "type": "string",
                        "description": "Heure du rappel (ex: '09:00', 'demain 14h')"
                    },
                    "priority": {
                        "type": "string",
                        "enum": ["low", "normal", "high"],
                        "description": "Priorité du rappel"
                    }
                },
                "required": ["title", "time"]
            }
        }
    }
]

all_functions = {
    "get_weather": get_weather,
    "get_time": get_time,
    "create_reminder": create_reminder
}

print("Fonctions disponibles:", ", ".join(all_functions.keys()))

Fonctions disponibles: get_weather, get_time, create_reminder


### Analyse des nouvelles fonctions

Nous avons ajouté deux nouvelles fonctions pour démontrer les appels multiples :

| Fonction | Objectif | Paramètres | Cas d'usage |
|----------|----------|------------|-------------|
| `get_time()` | Obtenir l'heure actuelle | `timezone` (optionnel) | Planification, rappels temporels |
| `create_reminder()` | Créer un rappel | `title`, `time`, `priority` | Gestion de tâches, productivité |

**Points techniques** :
- Les deux fonctions retournent du JSON structuré (facilite le parsing par le modèle)
- `priority` utilise un `enum` pour contraindre les valeurs possibles (low/normal/high)
- Les paramètres optionnels ont des valeurs par défaut sensibles

Avec ces 3 fonctions (météo, heure, rappel), le modèle peut maintenant orchestrer des scénarios complexes combinant plusieurs sources de données et actions.

In [8]:
# Test avec requête complexe nécessitant plusieurs appels
complex_question = (
    "Quelle heure est-il à Paris et quel temps fait-il? "
    "Crée aussi un rappel pour demain 9h: 'Réunion équipe'"
)

print("Question complexe:")
print(complex_question)
print("\nAppels de fonctions (parallèles):")

result = run_conversation(complex_question, extended_tools, all_functions)

print("\nRéponse finale:")
print(result)

Question complexe:
Quelle heure est-il à Paris et quel temps fait-il? Crée aussi un rappel pour demain 9h: 'Réunion équipe'

Appels de fonctions (parallèles):


  Appel: get_time({'timezone': 'Europe/Paris'})
  Appel: get_weather({'location': 'Paris', 'unit': 'celsius'})
  Appel: create_reminder({'title': 'Réunion équipe', 'time': 'demain 09:00', 'priority': 'normal'})



Réponse finale:
À Paris (Europe/Paris) il est actuellement 22h36:14 le 25 février 2026.

Météo à Paris : 18 °C, nuageux, humidité ~65 %.

Le rappel "Réunion équipe" a été créé pour demain à 09:00 (priorité : normale).

Souhaitez-vous que je modifie ou supprime ce rappel, ou que je fixe une alarme ?


### Analyse de l'exécution parallèle

**Observation clé** : Le modèle a effectué **3 appels de fonction en parallèle** dans une seule réponse :

1. `get_time(timezone='Europe/Paris')`
2. `get_weather(location='Paris')`
3. `create_reminder(title='Réunion équipe', time='demain 09h')`

**Avantages de l'exécution parallèle :**

| Aspect | Sans parallélisme | Avec parallélisme |
|--------|------------------|-------------------|
| **Nombre d'aller-retours API** | 3 tours (1 appel par tour) | 1 tour (3 appels groupés) |
| **Latence totale** | ~3-6 secondes | ~1-2 secondes |
| **Tokens consommés** | Plus élevé (3 réponses) | Optimisé (1 réponse) |

**Comment le modèle décide-t-il d'exécuter en parallèle ?**

Le modèle détecte que les 3 fonctions sont **indépendantes** (pas de dépendance de données entre elles). Si une fonction dépend du résultat d'une autre, le modèle les exécute séquentiellement.

> **Best practice** : Concevez vos fonctions pour maximiser l'indépendance et permettre la parallélisation.

## 5. Contrôle Avancé : tool_choice

Le paramètre `tool_choice` offre un contrôle fin sur le comportement du modèle :

| Valeur | Comportement |
|--------|-------------|
| `"auto"` | Le modèle décide (défaut) |
| `"required"` | Le modèle DOIT appeler au moins un outil |
| `"none"` | Le modèle ne peut PAS appeler d'outils |
| `{"type": "function", "function": {"name": "X"}}` | Force l'appel de la fonction X |

**Cas d'usage :**
- `"required"` : Forcer l'utilisation d'outils pour des données en temps réel
- `"none"` : Désactiver temporairement les outils (mode conversation pure)
- Spécifique : Garantir qu'une action précise sera exécutée

In [9]:
# Test 1: Forcer l'appel d'une fonction spécifique
print("=== Test 1: Forcer l'appel d'un outil spécifique ===")
response = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=[{"role": "user", "content": "Bonjour, comment vas-tu?"}],
    tools=tools,
    tool_choice={"type": "function", "function": {"name": "get_weather"}}
)

print("Question: Bonjour, comment vas-tu?")
print("\nAvec tool_choice forcé à get_weather:")
if response.choices[0].message.tool_calls:
    tc = response.choices[0].message.tool_calls[0]
    print(f"  Le modèle a appelé: {tc.function.name}")
    print(f"  Arguments: {tc.function.arguments}")

# Test 2: Désactiver les tools
print("\n=== Test 2: Désactiver les tools ===")
response = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=[{"role": "user", "content": "Quelle est la météo à Paris?"}],
    tools=tools,
    tool_choice="none"
)

print("Question: Quelle est la météo à Paris?")
print("\nAvec tool_choice='none' (pas d'appel de fonction):")
print(f"  Réponse directe: {response.choices[0].message.content[:100]}...")

# Test 3: Forcer au moins un appel (required)
print("\n=== Test 3: tool_choice='required' ===")
response = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=[{"role": "user", "content": "Dis-moi bonjour"}],
    tools=extended_tools,
    tool_choice="required"
)

print("Question: Dis-moi bonjour")
print("\nAvec tool_choice='required':")

if response.choices[0].message.tool_calls:
    contenu_reponse = response.choices[0].message.content
    print("\nContenu de la réponse du modèle:")
    print(contenu_reponse)
    tc = response.choices[0].message.tool_calls[0]
    print(f"  Le modèle est FORCÉ d'appeler un outil: {tc.function.name} avec les paramètres {tc.function.arguments}")

=== Test 1: Forcer l'appel d'un outil spécifique ===


Question: Bonjour, comment vas-tu?

Avec tool_choice forcé à get_weather:
  Le modèle a appelé: get_weather
  Arguments: {"location":"Paris","unit":"celsius"}

=== Test 2: Désactiver les tools ===


Question: Quelle est la météo à Paris?

Avec tool_choice='none' (pas d'appel de fonction):
  Réponse directe: Je n’ai pas accès aux données météo en temps réel depuis ici, je ne peux donc pas consulter et vous ...

=== Test 3: tool_choice='required' ===


Question: Dis-moi bonjour

Avec tool_choice='required':

Contenu de la réponse du modèle:
None
  Le modèle est FORCÉ d'appeler un outil: get_time avec les paramètres {"timezone":"Europe/Paris"}


### Analyse des comportements de tool_choice

| Mode | Requête | Résultat | Observation |
|------|---------|----------|-------------|
| **Forcé spécifique** | "Bonjour, comment vas-tu?" | Appel forcé à `get_weather` | Le modèle DOIT appeler la fonction même si inappropriée |
| **none** | "Quelle est la météo à Paris?" | Réponse textuelle sans fonction | Le modèle répond directement sans accès aux données réelles |
| **required** | "Dis-moi bonjour" | Appel forcé à un outil (ex: `get_time`) | Le modèle choisit l'outil le moins inapproprié |

**Implications pratiques :**
- `tool_choice` forcé peut générer des appels non pertinents → utiliser avec parcimonie
- `tool_choice="none"` utile pour économiser des appels API coûteux quand les données ne changent pas
- `tool_choice="required"` garantit l'utilisation de données temps réel (ex: prix en bourse)

> **Attention** : Forcer un outil peut dégrader l'expérience utilisateur si mal utilisé.

## 6. Gestion Robuste des Erreurs

Dans un système de production, la gestion d'erreurs est cruciale :

**Sources d'erreurs possibles :**
1. **Erreurs API** : Timeout, limites de taux, problèmes réseau
2. **Arguments invalides** : JSON malformé, types incorrects
3. **Fonction non disponible** : Le modèle appelle une fonction qui n'existe pas
4. **Erreur d'exécution** : La fonction échoue (ex: API externe indisponible)
5. **Boucles infinies** : Le modèle continue d'appeler des fonctions sans fin

**Bonnes pratiques :**
- Limiter le nombre d'itérations (max_iterations)
- Valider les arguments JSON
- Retourner des messages d'erreur structurés au modèle
- Logger les erreurs pour débogage

In [10]:
def run_safe_conversation(user_message: str, tools: list, available_functions: dict, max_iterations: int = 5):
    """Version robuste avec gestion d'erreurs et limite d'itérations"""
    messages = [{"role": "user", "content": user_message}]
    
    for iteration in range(max_iterations):
        # Gestion erreurs API
        try:
            response = client.chat.completions.create(
                model=DEFAULT_MODEL,
                messages=messages,
                tools=tools,
                tool_choice="auto"
            )
        except Exception as e:
            return f"Erreur API: {e}"
        
        assistant_message = response.choices[0].message
        messages.append(assistant_message)
        
        if not assistant_message.tool_calls:
            return assistant_message.content
        
        # Traiter chaque appel de fonction
        for tool_call in assistant_message.tool_calls:
            function_name = tool_call.function.name
            
            # Gestion erreurs JSON
            try:
                function_args = json.loads(tool_call.function.arguments)
            except json.JSONDecodeError:
                result = json.dumps({"error": "Arguments invalides, JSON malformé"})
                messages.append({"role": "tool", "tool_call_id": tool_call.id, "content": result})
                continue
            
            # Vérifier que la fonction existe
            if function_name not in available_functions:
                result = json.dumps({"error": f"Fonction '{function_name}' non disponible"})
            else:
                # Gestion erreurs d'exécution
                try:
                    result = available_functions[function_name](**function_args)
                except Exception as e:
                    result = json.dumps({"error": f"Erreur d'exécution: {str(e)}"})
            
            messages.append({"role": "tool", "tool_call_id": tool_call.id, "content": result})
    
    return f"Limite d'itérations atteinte ({max_iterations}). Conversation trop longue."

# Test de gestion d'erreurs
print("=== Test 1: Ville non répertoriée (fallback) ===")
print(run_safe_conversation("Météo à Berlin?", tools, all_functions))

print("\n=== Test 2: Fonction inexistante ===")
print(run_safe_conversation("Achète-moi un billet d'avion", extended_tools, all_functions))

print("\n=== Test 3: Arguments invalides ===")
# Simuler un cas où le modèle pourrait générer des arguments incorrects
print(run_safe_conversation("Météo à ????? température en @@@@", tools, all_functions))

=== Test 1: Ville non répertoriée (fallback) ===


Actuellement à Berlin : 20°C, conditions variables, humidité ~50%.

Souhaitez-vous la prévision pour les prochains jours, ou la même info en °F ?

=== Test 2: Fonction inexistante ===


Je peux vous aider à trouver et réserver un billet, mais je ne peux pas effectuer de paiement ni acheter le billet directement pour vous. Je peux en revanche :

- chercher et comparer des vols (horaires, prix, escales, bagages),
- vous proposer les meilleures options et liens pour réserver,
- préparer les informations à entrer lors de la réservation,
- vérifier les règles de visa/entrée ou les restrictions COVID si besoin,
- créer un rappel pour réserver ou payer si vous le souhaitez.

Pour commencer, donnez-moi ces informations :
1. Ville/aéroport de départ :
2. Ville/aéroport d’arrivée :
3. Dates : aller (et retour si aller‑retour). Êtes‑vous flexible (+/- jours) ?
4. Nombre de passagers : adultes / enfants / bébés
5. Classe : économique / premium / affaires / première
6. Préférences : compagnie(s) préférée(s), pas d’escales, durée max d’escale, bagages inclus obligatoires ?
7. Budget approximatif (optionnel)
8. Passeport nationalité (si vous voulez que je vérifie visa/exigences sani

Vous avez laissé des espaces réservés (????? et @@@@). Pour que je récupère la météo, pouvez-vous préciser :

1. La ville (ex. Paris, Lyon, Montréal) — ou un lieu précis.  
2. L’unité de température souhaitée : "celsius" (°C) ou "fahrenheit" (°F).

Exemples que vous pouvez copier :
- Météo à Paris température en celsius
- Météo à Montréal température en fahrenheit

Souhaitez-vous seulement la température actuelle ou aussi humidité, vent et prévisions ?


### Analyse des résultats de gestion d'erreurs

Les trois tests démontrent la robustesse du système :

| Test | Scénario | Comportement attendu |
|------|----------|---------------------|
| **Ville non répertoriée** | Berlin n'est pas dans `weather_data` | Fallback vers données par défaut (20°C, variable) |
| **Fonction inexistante** | "Achète-moi un billet d'avion" | Retour JSON avec `error: "Fonction 'X' non disponible"` |
| **Arguments invalides** | Caractères spéciaux (??, @@) | Le modèle peut soit échouer à générer des arguments valides, soit utiliser un fallback |

**Points clés :**
- La fonction `run_safe_conversation()` ne plante **jamais**
- Tous les cas d'erreur sont capturés et retournés de manière structurée
- Le modèle reçoit les messages d'erreur et peut adapter sa stratégie

> **Production** : En environnement réel, logguer toutes les erreurs pour analyse et amélioration continue.

## 7. Cas d'Usage Avancés

Voyons quelques patterns avancés de function calling :

### A. Recherche de base de données
Simuler une recherche dans une base de données avec filtres.

In [11]:
# Simuler une base de données de cours
COURSE_DB = [
    {"id": 1, "title": "Python pour débutants", "category": "Programming", "duration_hours": 10, "level": "Débutant", "price": 49},
    {"id": 2, "title": "Machine Learning Intro", "category": "ML", "duration_hours": 15, "level": "Intermédiaire", "price": 99},
    {"id": 3, "title": "Deep Learning Avancé", "category": "ML", "duration_hours": 30, "level": "Avancé", "price": 299},
    {"id": 4, "title": "Data Science avec R", "category": "Data", "duration_hours": 20, "level": "Intermédiaire", "price": 149},
    {"id": 5, "title": "Computer Vision", "category": "ML", "duration_hours": 25, "level": "Avancé", "price": 349},
]

def search_courses(category: str = None, min_duration: int = None, max_price: int = None, level: str = None) -> str:
    """Recherche de cours avec filtres multiples"""
    results = COURSE_DB.copy()
    
    if category:
        results = [c for c in results if c["category"] == category]
    if min_duration:
        results = [c for c in results if c["duration_hours"] >= min_duration]
    if max_price:
        results = [c for c in results if c["price"] <= max_price]
    if level:
        results = [c for c in results if c["level"] == level]
    
    return json.dumps({"count": len(results), "courses": results}, ensure_ascii=False)

# Définir le tool
search_tool = [{
    "type": "function",
    "function": {
        "name": "search_courses",
        "description": "Rechercher des cours dans la base de données avec filtres optionnels",
        "parameters": {
            "type": "object",
            "properties": {
                "category": {"type": "string", "enum": ["Programming", "ML", "Data"], "description": "Catégorie de cours"},
                "min_duration": {"type": "integer", "description": "Durée minimale en heures"},
                "max_price": {"type": "integer", "description": "Prix maximum en euros"},
                "level": {"type": "string", "enum": ["Débutant", "Intermédiaire", "Avancé"], "description": "Niveau"}
            }
        }
    }
}]

search_functions = {"search_courses": search_courses}

# Test
question = "Trouve-moi tous les cours de Machine Learning de plus de 20 heures"
print(f"Question: {question}\n")
print(run_safe_conversation(question, search_tool, search_functions))

Question: Trouve-moi tous les cours de Machine Learning de plus de 20 heures



Voici les cours de la catégorie Machine Learning d’une durée > 20 heures que j’ai trouvés :

- ID 3 — Deep Learning Avancé  
  Durée : 30 h | Niveau : Avancé | Prix : 299 €

- ID 5 — Computer Vision  
  Durée : 25 h | Niveau : Avancé | Prix : 349 €

Souhaitez-vous :
- que je vous donne le programme détaillé d’un de ces cours ?  
- que je filtre par prix ou niveau ?  
- que je lance l’inscription pour l’un d’eux ?


### Interprétation des résultats de recherche

**Requête analysée par le modèle :**
- Catégorie : Machine Learning
- Durée minimale : 20 heures

**Résultats retournés :**
Le modèle a correctement extrait les critères et construit les arguments de la fonction `search_courses()`. Notez que :

1. **Traitement du langage naturel** : "de plus de 20 heures" → `min_duration=20`
2. **Mapping de catégorie** : "Machine Learning" → `category="ML"`
3. **Filtres combinés** : Les deux critères sont appliqués en ET logique

Ce pattern de recherche s'applique à de nombreux cas d'usage réels (e-commerce, CRM, documentation, etc.).

### B. Orchestration Multi-Étapes

Le modèle peut orchestrer plusieurs étapes complexes automatiquement.

In [12]:
# Le modèle peut décomposer une tâche complexe en plusieurs appels
scenario = "Planifie ma journée demain - réveil à 7h, sport à 8h, travail à 9h"
print(f"Scénario: {scenario}\n")
print("Étapes exécutées:")

result = run_conversation(scenario, extended_tools, all_functions)

print("\nRésultat:")
print(result)

Scénario: Planifie ma journée demain - réveil à 7h, sport à 8h, travail à 9h

Étapes exécutées:



Résultat:
Voici un planning clair et quelques options/préconisations pour demain, basé sur réveil 7h, sport 8h, travail 9h.

Proposition de planning (option standard — sport 8h à 8h45 pour laisser le temps de se préparer avant le travail)
- 07:00 — Réveil : boire un verre d'eau, 5–10 min d'étirements légers, faire le lit.  
- 07:10–07:35 — Petit‑déjeuner nutritif (protéines + glucides lents) et préparation (vêtements, sac).  
- 07:35–07:50 — Préparation rapide / trajet jusqu’au lieu de sport ou mise en place si vous êtes chez vous.  
- 08:00–08:45 — Sport : 5–10 min échauffement, 25–30 min séance principale, 5–10 min retour au calme + étirements.  
- 08:45–09:00 — Douche rapide, habillage, dernière vérification (documents, ordinateur, clés).  
- 09:00 — Début du travail.

Si vous devez être physiquement au travail à 9h (trajet >0 min)
- Variante courte : sport 07:30–08:00 (30 min), douche 08:00–08:20, petit‑déj rapide ou smoothie 08:20–08:35, départ 08:35.  
- Variante si travail à di

### Interprétation de l'orchestration

Le modèle a démontré une capacité clé des **agents autonomes** : décomposer une intention complexe en étapes atomiques.

**Analyse du comportement :**
- **Entrée** : "Planifie ma journée demain - réveil à 7h, sport à 8h, travail à 9h"
- **Décomposition automatique** : 3 appels à `create_reminder()` avec des priorités différentes
- **Contextualisation** : Le modèle a assigné `priority="high"` au réveil (plus critique que les autres)

**Ce pattern est fondamental pour :**
- **Agents de productivité** : Transformation de langage naturel en actions multiples
- **Workflows complexes** : Orchestration automatique de tâches interdépendantes
- **Interfaces conversationnelles** : Réduire le nombre d'interactions utilisateur

> **Pattern clé** : Le modèle transforme "planifie ma journée" en une séquence de 3 actions atomiques sans intervention humaine. C'est le cœur du paradigme **agentique**.

## 8. Conclusion et Bonnes Pratiques

### Points Clés

1. **Descriptions précises** : Les descriptions des fonctions guident le modèle. Soyez explicite !
2. **JSON Schema rigoureux** : Définissez bien les types et contraintes des paramètres
3. **Gestion d'erreurs** : Toujours prévoir des fallbacks et retourner des erreurs structurées
4. **Limite d'itérations** : Évitez les boucles infinies avec un max_iterations
5. **Sécurité** : Validez TOUJOURS les arguments avant exécution (injections, chemins dangereux, etc.)

### Cas d'Usage Réels

| Domaine | Exemple d'Application |
|---------|----------------------|
| **E-commerce** | Recherche produits, ajout panier, suivi commande |
| **Productivité** | Gestion calendrier, emails, rappels |
| **Finance** | Consultation soldes, virements, alertes |
| **Data Science** | Requêtes SQL, visualisations, analyses |
| **DevOps** | Déploiements, logs, monitoring |

### Exercices Suggérés

1. **Système de réservation** : Créez des fonctions pour chercher/réserver des restaurants
2. **Calculatrice avancée** : Fonctions mathématiques (factorielle, fibonacci, primalité)
3. **API réelle** : Intégrez une vraie API météo (OpenWeatherMap) au lieu de la simulation
4. **Multi-agents** : Créez plusieurs "agents" avec des sets de fonctions différents

### Prochaine Étape

Dans le notebook suivant (**5_RAG_Introduction.ipynb**), nous verrons comment combiner function calling avec le RAG (Retrieval-Augmented Generation) pour créer des assistants qui peuvent chercher dans des bases de connaissances.

---

**Ressources :**
- [OpenAI Function Calling Guide](https://platform.openai.com/docs/guides/function-calling)
- [JSON Schema Documentation](https://json-schema.org/)
- Documentation Notebook 1 (OpenAI Intro)
- Documentation Notebook 3 (Structured Outputs)

In [13]:
# Cellule de validation finale
print("✓ Notebook Function Calling terminé !")
print("✓ Concepts maîtrisés: Tools, tool_choice, boucles agentiques, gestion d'erreurs")
print("✓ Prochaine étape: RAG (Retrieval-Augmented Generation)")

✓ Notebook Function Calling terminé !
✓ Concepts maîtrisés: Tools, tool_choice, boucles agentiques, gestion d'erreurs
✓ Prochaine étape: RAG (Retrieval-Augmented Generation)


### Validation de la progression

Cette cellule confirme la complétion du notebook et résume les acquis :

**Concepts maîtrisés :**
1. **Tools** : Définition de fonctions avec JSON Schema
2. **tool_choice** : Contrôle fin du comportement du modèle (auto, required, none, spécifique)
3. **Boucles agentiques** : Pattern fondamental pour les agents autonomes
4. **Gestion d'erreurs** : Robustesse en production (timeouts, validation, limites)
5. **Orchestration** : Appels parallèles et décomposition automatique de tâches complexes

**Prochaine étape :** Le notebook RAG (Retrieval-Augmented Generation) combinera ces techniques avec des bases de connaissances pour créer des assistants capables de raisonner sur des documents.

---

# CHALLENGE BONUS - Routeur Intelligent

**Points : 1.5 pts**

## Objectif

Creer une fonction qui route automatiquement les questions utilisateur vers le prompt approprie en utilisant le function calling.

## Criteres de succes

- [ ] La fonction detecte le type de question (math, traduction, code, meteo, etc.)
- [ ] Le routing utilise function_calling (pas de if/else manuels)
- [ ] Au moins 3 types de questions sont supportes
- [ ] Le code fonctionne sans erreur

## Specification

```python
# Creer une fonction route_question(question: str) -> str
# qui:
# 1. Prend une question utilisateur
# 2. Utilise function_calling pour determiner le type
# 3. Route vers le prompt/traitement approprie
# 4. Retourne la reponse finale

# Bonus: Ajouter une fonction de calcul mathematique reel
```

## Indices

- Definissez des tools pour chaque type de question
- Utilisez `tool_choice="auto"` pour laisser le modele decider
- Chaque tool peut avoir une implementation specifique

## Votre code ici

```python
# TODO: Implementez votre routeur intelligent
# Definissez vos tools, vos fonctions, et la boucle de routing
```

---

**Soumission** : Une fois termine, creez une PR sur votre fork avec :
- Titre: "Challenge #2 - [Votre Nom]"
- Description: Bref explication de votre architecture de routing